In [1]:
#| default_exp learner

In [2]:
#|export
import pickle,gzip,math,os,time,shutil,torch,matplotlib as mpl,numpy as np,matplotlib.pyplot as plt
import fastcore.all as fc
from collections.abc import Mapping
from pathlib import Path
from operator import attrgetter,itemgetter
from functools import partial

from torch import tensor,nn,optim
from torch.utils.data import DataLoader,default_collate
import torch.nn.functional as F
from datasets import load_dataset,load_dataset_builder

from miniai.datasets import *

from fastprogress import progress_bar

In [3]:
import torchvision.transforms.functional as TF
from fastcore.test import test_close

torch.set_printoptions(precision=2, linewidth=140, sci_mode=False)
torch.manual_seed(1)
mpl.rcParams['image.cmap'] = 'gray'

import logging
logging.disable(logging.WARNING)

In [4]:
# from torch.utils.data.sampler import BatchSampler, RandomSampler, SequentialSampler
# from transformers import default_data_collator
# from collections.abc import Mapping

## Learner

In [5]:
x,y = 'image','label'
name = "fashion_mnist"
dsd = load_dataset(name)

  0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
@inplace
def transformi(b): b[x] = [torch.flatten(TF.to_tensor(o)) for o in b[x]]

In [7]:
bs = 256
tds = dsd.with_transform(transformi)

In [8]:
#|export
class DataLoaders:
    def __init__(self, *dls): self.train,self.valid = dls[:2]

    @classmethod
    def from_dd(cls, dd, batch_size, as_tuple=True):
        return cls(*[DataLoader(ds, batch_size, collate_fn=collate_dict(ds)) for ds in dd.values()])

In [9]:
dls = DataLoaders.from_dd(tds, bs)
dt = dls.train
xb,yb = next(iter(dt))
xb.shape,yb[:10]

(torch.Size([256, 784]), tensor([9, 0, 0, 3, 0, 2, 7, 2, 5, 5]))

In [10]:
#|export
def to_cuda(x):
    if isinstance(x, Mapping): return {k:v.cuda() for k,v in x.items()}
    return type(x)(o.cuda() for o in x)

In [11]:
#|export
class Learner:
    def __init__(self, model, dls, loss_func, lr, opt_func=optim.SGD): fc.store_attr()

    def one_batch(self):
        self.xb,self.yb = to_cuda(self.batch)
        self.preds = self.model(self.xb)
        self.loss = self.loss_func(self.preds, self.yb)
        if self.model.training:
            self.loss.backward()
            self.opt.step()
        with torch.no_grad(): self.calc_stats()
    
    def calc_stats(self):
        acc = (self.preds.argmax(dim=1)==self.yb).float().sum()
        self.accs.append(acc)
        n = len(self.xb)
        self.losses.append(self.loss*n)
        self.ns.append(n)

    def one_epoch(self, train):
        self.model.training = train
        dl = self.dls.train if train else self.dls.valid
        for self.num,self.batch in enumerate(dl): self.one_batch()
        n = sum(self.ns)
        print(self.epoch, self.model.training, sum(self.losses).item()/n, sum(self.accs).item()/n)
    
    def fit(self, n_epochs):
        self.accs,self.losses,self.ns = [],[],[]
        model.cuda()
        self.opt = self.opt_func(self.model.parameters(), self.lr)
        self.n_epochs = n_epochs
        for self.epoch in range(n_epochs):
            self.one_epoch(True)
            self.one_epoch(False)

In [12]:
m,nh = 28*28,50
model = nn.Sequential(nn.Linear(m,nh), nn.ReLU(), nn.Linear(nh,10))

In [13]:
learn = Learner(model, dls, F.cross_entropy, lr=0.001)
learn.fit(1)

0 True 1.3609341145833334 0.54695
0 False 1.3473861607142856 0.5684714285714285


## Metrics

In [14]:
#|export
class Metric:
    def __init__(self): self.reset()
    def reset(self): self.inps,self.targs,self.ns = [],[],[]
    def add(self, inp, targ=None, n=1):
        self.inps.append(inp)
        if targ is not None: self.targs.append(targ)
        self.ns.append(n)
    def add_batch(self, inps, targs):
        for inp,targ in zip(inps,targs): self.add(inp,targ)
    def __call__(self, inps=None, targs=None, ns=None):
        inps = self.inps if inps is None else inps
        targs = self.targs if targs is None else targs
        if ns is None: ns = self.ns
        if ns is None: ns = [1]*len(inps)
        return self.forward(tensor(inps), tensor(targs) if targs else None, tensor(ns))
    def forward(self, inps, targs, ns): raise NotImplementedError()

In [15]:
#|export
class Accuracy(Metric):
    def forward(self, inps, targs, ns):
        res = (inps==targs).float()*ns
        return res.sum()/ns.sum()

In [16]:
acc = Accuracy()
acc.add_batch([0, 1, 2, 0, 1, 2], [0, 1, 1, 2, 1, 0])
acc()

tensor(0.50)

In [17]:
#|export
class Loss(Metric):
    def forward(self, inps, targs, ns): return (inps*ns).sum()/ns.sum()

In [18]:
loss = Loss()
loss.add(0.6, n=32)
loss.add(0.9, n=2)
loss()

tensor(0.62)

## Callbacks learner

In [24]:
#|export
def identity(*args):
    if not args: return
    x,*args = args
    return (x,)+args if args else x

In [25]:
#|export
class CancelFitException(Exception): pass
class CancelBatchException(Exception): pass
class CancelEpochException(Exception): pass

In [26]:
#|export
class with_cbs:
    def __init__(self, nm): self.nm = nm
    def __call__(self, f):
        def _f(o, *args, **kwargs):
            try:
                o.callback(f'before_{self.nm}')
                f(o, *args, **kwargs)
                o.callback(f'after_{self.nm}')
            except globals()[f'Cancel{self.nm.title()}Exception']: pass
        return _f

In [27]:
#|export
class Learner():
    def __init__(self, model, dls, loss_func, lr, cbs, opt_func=optim.SGD):
        fc.store_attr()
        for cb in cbs: cb.learn = self

    @with_cbs('batch')
    def one_batch(self):
        self.predict()
        self.get_loss()
        if self.model.training:
            self.backward()
            self.step()

    def one_epoch(self, train):
        self.model.train(train)
        self.dl = self.dls.train if train else self.dls.valid
        self._one_epoch()

    @with_cbs('epoch')
    def _one_epoch(self):
        for self.iter,self.batch in enumerate(self.dl): self.one_batch()
    
    def fit(self, n_epochs):
        self.n_epochs = n_epochs
        self.epochs = range(n_epochs)
        self.opt = self.opt_func(self.model.parameters(), self.lr)
        self._fit()

    @with_cbs('fit')
    def _fit(self):
        for self.epoch in self.epochs:
            self.one_epoch(True)
            self.one_epoch(False)
        
    def __getattr__(self, name):
        if name[0]=='_': raise AttributeError(name)
        return partial(self.callback, name)

    def callback(self, method_nm):
        for cb in sorted(self.cbs, key=attrgetter('order')): getattr(cb, method_nm,identity)()

In [28]:
#|export
class Callback(): order = 0

In [29]:
#|export
class TrainCB(Callback):
    def predict(self): self.learn.preds = self.learn.model(self.learn.batch[0])
    def get_loss(self): self.learn.loss = self.learn.loss_func(self.learn.preds, self.learn.batch[1])
    def backward(self): self.learn.loss.backward()
    def step(self): self.learn.opt.step()            

In [30]:
#|export
class CudaCB(Callback):
    def before_fit(self): self.learn.model.cuda()
    def before_batch(self): self.learn.batch = to_cuda(self.learn.batch)

In [31]:
#|export
class ProgressCB(Callback):
    def before_epoch(self): self.learn.dl = progress_bar(self.learn.dl)

In [32]:
#|export
class MetricsCB(Callback):
    def __init__(self, metric): self.metric,self.loss_m = metric,Loss()
    def before_epoch(self):
        self.metric.reset()
        self.loss_m.reset()
    def after_epoch(self): print(self.learn.epoch, self.learn.model.training, self.metric(), self.loss_m())
    def after_batch(self):
        self.metric.add_batch(self.learn.preds.argmax(dim=1), self.learn.batch[1])
        self.loss_m.add(self.learn.loss, len(self.learn.batch[0]))

In [33]:
m,nh = 28*28,50
model = nn.Sequential(nn.Linear(m,nh), nn.ReLU(), nn.Linear(nh,10))

In [34]:
cbs = [TrainCB(), CudaCB(), MetricsCB(Accuracy()), ProgressCB()]
learn = Learner(model, dls, F.cross_entropy, lr=0.001, cbs=cbs)
learn.fit(1)

0 True tensor(0.55) tensor(1.33)


0 False tensor(0.70) tensor(1.11)


### TrainingLearner subclass

In [35]:
#|export
class TrainingLearner(Learner):
    def predict(self): self.preds = self.model(self.batch[0])
    def get_loss(self): self.loss = self.loss_func(self.preds, self.batch[1])
    def backward(self): self.loss.backward()
    def step(self): self.opt.step()            

In [36]:
# NB: No TrainCB
cbs = [CudaCB(), MetricsCB(Accuracy()), ProgressCB()]
learn = TrainingLearner(model, dls, F.cross_entropy, lr=0.001, cbs=cbs)
learn.fit(1)

0 True tensor(0.75) tensor(1.23)


0 False tensor(0.73) tensor(1.69)


## Export -

In [30]:
import nbdev; nbdev.nbdev_export()